In [48]:
from ipyleaflet import Map, Marker, MarkerCluster, AwesomeIcon, Popup, basemaps, basemap_to_tiles
from ipywidgets import HTML

import pandas as pd

In [49]:
# methods for hospital layer

def get_popup_html_message_for_hospital(df, values, names):
    text = """
        <!DOCTYPE html>
        <html>
        <head>
        <style>
        table {
          font-family: arial, sans-serif;
          border-collapse: collapse;
          width: 100%;
        }

        td, th {
          border: 1px solid #999999;
          text-align: left;
          padding: 2px;
        }

        tr:nth-child(even) {
          background-color: #eeeeee;
        }
        </style>
        </head>
        <body>

        <h4>""" + name + """</h4>

        <table>
    """
    for d in df:
        
  <tr>
    <td>Вкупно</td>
    <td>""" + str(count_all) + """</td>
  </tr>
  <tr>
    <td>Заболени</td>
    <td>""" + str(count_sick) + """</td>
  </tr>
  <tr>
    <td>Излечени</td>
    <td>""" + str(count_healthy) + """</td>
  </tr>
  <tr>
    <td>Мртви</td>
    <td>""" + str(count_dead) + """</td>
  </tr>
    text +="""
    </table>

    </body>
    </html>
    """
    message = HTML()
    message.value = text
    return message
    
def get_popup_for_hospital(message):
    popup = Popup(
        child=message,
        close_button=True,
        auto_close=False,
        close_on_escape_key=False
    )
    return popup

def add_hospital_layer(m):
    hospital_data = pd.read_csv("podatoci/AdditionalInfo-Hospitals.csv")
    
    hospital_icon = AwesomeIcon(
        name='h-square',
        marker_color='red',
        icon_color='white',
        spin=False
    )
    
    all_patients = pd.read_csv("podatoci/Корона вирус податоци Македонија - Хоспитализирани.csv", header=0,
                 names=['hospital', "date", "count", "age_range"]).groupby(["hospital"]).sum()

    recovered = pd.read_csv("podatoci/Корона вирус податоци Македонија - Оздравени по Болница.csv", header=0,
                     names=['hospital', "date", "count", "age_range"]).groupby(["hospital"]).sum()

    dead = pd.read_csv("podatoci/Корона вирус податоци Македонија - Мртви по Болница.csv", header=0,
                     names=['hospital', "date", "count", "age_range"]).groupby(["hospital"]).sum()

    
    hospital_list = list()

    for ind in hospital_data.index:
        hospital_name = hospital_data["Hospital"][ind]
        location = (hospital_data["Longitude"][ind], hospital_data["Latitude"][ind])
        marker = Marker(location=location, draggable=False, icon=hospital_icon)
        
        count_all = all_patients.loc[hospital_name]["count"] if hospital_name in all_patients.index else 0
        count_recovered = recovered.loc[hospital_name]["count"] if hospital_name in recovered.index else 0
        count_dead = dead.loc[hospital_name]["count"] if hospital_name in dead.index else 0
        count_sick = count_all - (count_recovered + count_dead)
        
        marker.popup = get_popup_for_hospital(get_popup_html_message_for_hospital(hospital_name, count_all, count_sick, count_recovered, count_dead))
        hospital_list.append(marker)



    hospitals = MarkerCluster(markers=tuple(hospital_list))
    m.add_layer(hospitals)

In [102]:
city_df = pd.read_csv("../podatoci/mkcities.csv")
city_df['city'] = city_df.city_cyrilic
city_df

,id,city_cyrilic,admin_name_cyrilic,city_latin,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population
0,1807600615,Скопје,Скопје,Skopje,Скопје,Skopje,42.0000,21.4335,Macedonia,MK,MKD,Skopje,False,494087.0
1,1807020286,Тетово,Тетово,Tetovo,Тетово,Tetovo,42.0092,20.9701,Macedonia,MK,MKD,Tetovo,False,119132.0
2,1807488567,Битола,Битола,Bitola,Битола,Bitola,41.0391,21.3395,Macedonia,MK,MKD,Bitola,False,86528.0
3,1807691189,Сопиште,Сопиште,Sopište,Сопиште,Sopiste,41.9547,21.4275,Macedonia,MK,MKD,Sopište,False,NaN
4,1807895120,Кичево,Кичево,Kičevo,Кичево,Kicevo,41.5127,20.9589,Macedonia,MK,MKD,Kičevo,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,1807953626,Белишта,Дебарца,Belčišta,Белишта,Belcista,41.3028,20.8303,Macedonia,MK,MKD,Debarca,False,NaN
67,1807782793,Старо Нагоричане,Старо Нагоричане,Staro Nagoričane,Старо Нагоричане,Staro Nagoricane,42.1981,21.8286,Macedonia,MK,MKD,Staro Nagoričane,False,NaN
68,1807831162,Македонски Брод,Македонски Брод,Makedonski Brod,Македонски Брод,Makedonski Brod,41.5136,21.2153,Macedonia,MK,MKD,Makedonski Brod,False,NaN
69,1807936735,Новаци,Новаци,Novaci,Новаци,Novaci,41.0420,21.4587,Macedonia,MK,MKD,Novaci,False,NaN


In [77]:
infected_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=0&single=true&output=csv", 
                          names=['city', 'infected_in', 'date', 'count', 'age'], header=0)

infected_df

,city,infected_in,date,count,age
0,Скопје,Италијa,2020-02-26,1,50-59
1,Дебар,Италија,2020-03-06,2,60-69
2,Дебар,Македонија,2020-03-09,2,60-69
3,Скопје,Италија,2020-03-09,1,50-59
4,Дебар,Македонија,2020-03-09,1,40-49
5,Дебар,Македонија,2020-03-11,2,60-69
6,Дебар,Македонија,2020-03-13,4,50-59
7,Скопје,Шпанија,3/14/2020,1,непознато
8,Дебар,Македонија,3/14/2020,5,непознато


In [ ]:
healed_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=757629356&single=true&output=csv",
                       names=['city', 'date', 'count', 'age'], header=0)
healed_df

In [ ]:
dead_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=1754038661&single=true&output=csv",
                                               header=0, names=['city', 'date', 'count', 'age'])
dead_df

In [ ]:
infected_by_city = infected_df[['city', 'count']].copy().groupby('city').sum()
healded_by_city = healed_df[['city', 'count']].copy().groupby('city').sum()
dead_by_city = dead_df[['city', 'count']].copy().groupby('city').sum()
m = pd.merge(infected_by_city, healded_by_city, on='city', how='outer')
m = pd.merge(m, dead_by_city, on='city', how='outer')
m

In [ ]:
m.fillna(0, inplace=True)
m = pd.merge(m, city_df, on='city', how='inner')
m

In [ ]:


center=(41.6086, 21.7453)

m = Map(
    layers=(basemap_to_tiles(basemaps.OpenStreetMap.Mapnik, "2017-04-08"), ),
    center=center,
    zoom=8
)

add_hospital_layer(m)

m

,city,date,count,age


In [79]:
dead_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=1754038661&single=true&output=csv",
                                               header=0, names=['city', 'date', 'count', 'age'])
dead_df

,city,date,count,age


In [105]:
infected_by_city = infected_df[['city', 'count']].copy().groupby('city').sum()
healded_by_city = healed_df[['city', 'count']].copy().groupby('city').sum()
dead_by_city = dead_df[['city', 'count']].copy().groupby('city').sum()
m = pd.merge(infected_by_city, healded_by_city, on='city', how='outer')
m = pd.merge(m, dead_by_city, on='city', how='outer')
m

,count_x,count_y,count
city,,,
Дебар,16,NaN,NaN
Скопје,3,NaN,NaN


In [106]:
m.fillna(0, inplace=True)
m = pd.merge(m, city_df, on='city', how='inner')
m

,city,count_x,count_y,count,id,city_cyrilic,admin_name_cyrilic,city_latin,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population
0,Дебар,16,0.0,0.0,1807621851,Дебар,Дебар,Debar,Debar,41.5244,20.5242,Macedonia,MK,MKD,Debar,False,NaN
1,Скопје,3,0.0,0.0,1807600615,Скопје,Скопје,Skopje,Skopje,42.0000,21.4335,Macedonia,MK,MKD,Skopje,False,494087.0


In [75]:


center=(41.6086, 21.7453)

m = Map(
    layers=(basemap_to_tiles(basemaps.OpenStreetMap.Mapnik, "2017-04-08"), ),
    center=center,
    zoom=8
)

add_hospital_layer(m)

m

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/aleks/anaconda3/envs/jupyter/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-75-077095d10e47>", line 9, in <module>
    add_hospital_layer(m)
  File "<ipython-input-49-1a22c1239abf>", line 68, in add_hospital_layer
    hospital_data = pd.read_csv("podatoci/AdditionalInfo-Hospitals.csv")
  File "/home/aleks/anaconda3/envs/jupyter/lib/python3.7/site-packages/pandas/io/parsers.py", line 685, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "/home/aleks/anaconda3/envs/jupyter/lib/python3.7/site-packages/pandas/io/parsers.py", line 457, in _read
    parser = TextFileReader(fp_or_buf, **kwds)
  File "/home/aleks/anaconda3/envs/jupyter/lib/python3.7/site-packages/pandas/io/parsers.py", line 895, in __init__
    self._make_engine(self.engine)
  File "/home/aleks/anaconda3/envs/jupyter/lib/python3.7/site-packages/pandas/

FileNotFoundError: [Errno 2] File b'podatoci/AdditionalInfo-Hospitals.csv' does not exist: b'podatoci/AdditionalInfo-Hospitals.csv'

NameError: name 'hospital_data' is not defined